# Constructing Circuit Representation

In this section, we explain how to construct quantum circuits with the "Circuit Representation".

## Creating an empty circuit

To construct a quantum circuit in the circuit representation, you first need to create an instance of the {py:class}`~mqc3.circuit.CircuitRepr` class.
You can create an instance `c` with the name "empty" as shown below:

In [ ]:
from mqc3.circuit import CircuitRepr

c = CircuitRepr("empty")

The assigned name can be retrieved using the {py:attr}`~mqc3.circuit.CircuitRepr.name` property:

In [ ]:
print(c.name)

## Adding modes

When a circuit instance is created, it contains no modes or gates.
You can add modes using {py:meth}`~mqc3.circuit.CircuitRepr.Q`.
A newly added mode is initialized in the $x$-squeezed state, which serves as an input to the machinery of MQC3, as explained in {ref}`sec:theory`.

In [ ]:
c = CircuitRepr("create_mode")
print("#modes =", c.n_modes)
c.Q(0)  # Create a mode 0.
print("#modes =", c.n_modes)

Since modes are indexed starting from `0`, creating a mode with id=`N` implicitly creates modes with IDs ranging from `0` to `N-1`.

In [ ]:
c = CircuitRepr("create_modes")
print("#modes =", c.n_modes)
c.Q(9)  # Create a mode with id=9 implicitly creates modes with IDs 0 to 8 as well.
print("#modes =", c.n_modes)

## Adding gates

The available gates are defined in the {py:mod}`mqc3.circuit.ops.intrinsic` and {py:mod}`mqc3.circuit.ops.std` modules.

* The {py:mod}`~mqc3.circuit.ops.intrinsic` module defines gates that can be executed efficiently on the machinery of MQC3.
* The {py:mod}`~mqc3.circuit.ops.std` module provides commonly used gates in general quantum computing. These gates are implemented using intrinsic gates and are expanded into an array of intrinsic gates when the circuit is executed.

You can add gates to a circuit by specifying the modes to which they apply.
For instance, you can apply the {py:class}`~mqc3.circuit.ops.intrinsic.PhaseRotation` with $\phi = \pi$ to mode `0` of `c` as shown below:

![single mode gate](_images/circuit_repr_single_mode_gate.svg)

In [ ]:
# pyright: reportUnusedExpression=false

from math import pi

from mqc3.circuit.ops.intrinsic import PhaseRotation

c = CircuitRepr("single_mode_gate")
c.Q(0) | PhaseRotation(phi=pi)

c

You can apply multiple gates to the same mode by chaining.
For example,

* `c.Q(0) | PhaseRotation(phi=pi / 2)` and `c.Q(0) | Displacement(1, 1)` apply two gates separately.
* `c.Q(0) | PhaseRotation(phi=pi / 2) | Displacement(1, 1)` chains them together in a single statement.

![chain of single mode gates](_images/circuit_repr_single_mode_gate_chain.svg)

In [ ]:
# pyright: reportUnusedExpression=false

from mqc3.circuit.ops.intrinsic import Displacement

c = CircuitRepr("chain_of_single_mode_gates")
c.Q(0) | PhaseRotation(phi=pi / 2) | Displacement(1, 1)

c

For two-mode gates, you need to specify both modes (in this case, mode `0` and mode `1`):

![two-mode gate](_images/circuit_repr_two_mode_gate.svg)

In [ ]:
# pyright: reportUnusedExpression=false

from mqc3.circuit.ops.intrinsic import ControlledZ

c = CircuitRepr("two_mode_gate")
c.Q(0, 1) | ControlledZ(g=1)

c

For each mode, gates are applied in the order they are written.

```{caution}
In circuit representations, it is assumed that no operation is applied to a mode once it has been measured. The behavior is undefined if any operation is applied after {py:class}`mqc3.circuit.ops.intrinsic.Measurement` operation.
```

The following example demonstrates a circuit with multiple gates:

![sample circuit](_images/circuit_repr_sample_circuit.svg)

In [ ]:
# pyright: reportUnusedExpression=false

from math import pi

from mqc3.circuit import CircuitRepr
from mqc3.circuit.ops.intrinsic import ControlledZ, Displacement, Measurement, PhaseRotation

c = CircuitRepr("multiple_gates")
c.Q(0) | PhaseRotation(phi=pi / 4) | Displacement(1, -1)
c.Q(0, 1) | ControlledZ(g=1)
c.Q(0) | Measurement(theta=0)
c.Q(1) | Measurement(theta=pi / 2)

c

You can verify the gates have been added correctly by printing the circuit information:

In [ ]:
print("name        =", c.name)
print("#modes      =", c.n_modes)
print("#operations =", c.n_operations)
print(c)

(sec:circuit-adding-feedforward)=

## Adding feedforward

In MQC3, the parameters of certain operations can depend on the measurement results of other modes.
This mechanism is referred to as **feedforward** in MQC3.

To use feedforward, follow these steps:

1. Define a feedforward function `f` ({py:class}`~mqc3.feedforward.FeedForwardFunction`)
    * Create a Python function that takes a float as input and returns a float.
    * Decorate the function with {py:deco}`~mqc3.feedforward.feedforward`.
    * **Note:** The function must be **self-contained** and able to run independently.
        * If it depends on external modules such as `math` or `numpy`, import them **inside** the function.
2. Obtain the measurement result variable `m` ({py:class}`~mqc3.circuit.program.MeasuredVariable`)
3. Use the feedforwarded parameter `f(m)` as a parameter of an operation.

For example, the teleportation circuit shown below can be implemented as follows:

![teleportation](_images/circuit_repr_teleportation.svg)

In [ ]:
# pyright: reportUnusedExpression=false

from math import pi

from mqc3.circuit import CircuitRepr
from mqc3.circuit.ops.intrinsic import Displacement, Measurement
from mqc3.circuit.ops.std import BeamSplitter
from mqc3.feedforward import feedforward


# Define feedforward functions.
@feedforward
def displace_x(x):
    from math import sqrt  # noqa:PLC0415

    return sqrt(2) * x


@feedforward
def displace_p(p):
    from math import sqrt  # noqa:PLC0415

    return -sqrt(2) * p


# Construct the teleportation circuit.
c = CircuitRepr("teleportation")
c.Q(1, 2) | BeamSplitter(theta=pi / 4, phi=pi)
c.Q(0, 1) | BeamSplitter(theta=pi / 4, phi=pi)
# Measure modes 0 and 1.
m0 = c.Q(0) | Measurement(theta=pi / 2)
m1 = c.Q(1) | Measurement(theta=0)
# Apply displacement with feedforward.
c.Q(2) | Displacement(displace_x(m0), displace_p(m1))

c

The created circuit can be visualized using the {py:func}`~mqc3.circuit.visualize.make_figure` function.
For more details, see [Visualizing Circuit Representation](viz_circuit_repr.ipynb).

In [ ]:
from mqc3.circuit.visualize import make_figure

make_figure(c);

## Initializing modes

The {py:class}`~mqc3.client.MQC3Client` executes quantum circuits on MQC3's optical quantum computer, with the initial state being partially configurable via the specified squeezing angles.
In contrast, the {py:class}`~mqc3.client.SimulatorClient` executes quantum circuits in a circuit representation simulator, allowing the initial state to be set freely.

The initial state of a mode can be set using the {py:meth}`~mqc3.circuit.CircuitRepr.set_initial_state` method.
For {py:class}`~mqc3.client.MQC3Client` execution, the initial state must be an instance of {py:class}`~mqc3.circuit.state.HardwareConstrainedSqueezedState` representing a squeezed state with hardware constraints.
For {py:class}`~mqc3.client.SimulatorClient` execution, the initial state must be an instance of {py:class}`~mqc3.circuit.state.BosonicState` representing a superposition of Gaussian states ({py:class}`~mqc3.circuit.state.GaussianState`).
For an example usage, see {ref}`sec:simulation-configuring-circuit-representation`.

```{seealso}
See {ref}`this section<sec:execution-resource-and-initialized-states>` (for {py:class}`~mqc3.client.MQC3Client` execution) and {ref}`this section<sec:simulation-resource-and-initialized-states>` (for {py:class}`~mqc3.client.SimulatorClient` execution) for initial state details.
```